# 🧪 Notebook 1: Visualizing Molecular Geometries and Energy Scans

## Introduction: Exploring Excited-State PESs with Machine Learning

In this tutorial series, we explore how machine learning (ML) models can be trained to describe excited-state potential energy surfaces (PESs) and non-adiabatic couplings (NACs). As a test case, we use the amino methyl cation, 
$\mathrm{CH_2NH_2^+}$, a small system that nonetheless captures many of the essential features of photochemical reactivity.

### Why $\mathrm{CH_2NH_2^+}$?

$\mathrm{CH_2NH_2^+}$ is a prototypical molecule in photochemistry and non-adiabatic dynamics. It exhibits strong coupling between electronic states and undergoes ultrafast internal conversion processes. These characteristics make it a popular benchmark system for methods that go beyond the Born–Oppenheimer approximation. Importantly, its small size makes it computationally manageable while still containing enough electronic structure complexity to pose a challenge for both traditional and machine-learned approaches.

### What we will do:

In the first notebook, we load and visualize the molecular structure of $\mathrm{CH_2NH_2^+}$ along with a collection of geometries that span different regions of its configuration space. These geometries represent snapshots that the molecule might explore during photoinduced dynamics.

In [ ]:
import ase
from ase.db import connect
import nglview
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm

### Inspecting the geometries

In [ ]:
BOHR_TO_ANGSTROM = 0.529177249

In [ ]:
db = connect('grid_configuration.db')

In [ ]:
# We need to convert the distance from the database, which is in atomic units, 
# to Angstrom to be able to visualize the structures properly

traj_in_angstrom = []
number_of_frames = len(db)
for idx in tqdm(range(number_of_frames)):
    atoms = db.get_atoms(idx+1)
    atoms.set_positions(atoms.get_positions() * BOHR_TO_ANGSTROM)
    traj_in_angstrom.append(atoms)

In [ ]:
nglview.show_asetraj(traj_in_angstrom)

In [ ]:
traj_in_angstrom[0].get_chemical_symbols()

## Inspecting the PES

🌀 Where to Expect Conical Intersections in $\mathrm{CH_2NH_2^+}$
 
One of the defining features of excited-state dynamics is the presence of conical intersections (CIs)—points where two adiabatic potential energy surfaces become degenerate and the Born–Oppenheimer approximation breaks down. These regions allow for ultrafast nonradiative transitions between electronic states and often govern the outcome of photochemical processes.

In the case of $\mathrm{CH_2NH_2^+}$, conical intersections typically occur in distorted geometries, especially along coordinates that involve:
- C–N bond stretching
- Twisting around the C–N bond
- Out-of-plane pyramidalization of the carbon or nitrogen atom

These distortions break the planarity and symmetry of the molecule, allowing the $S_1$ and $S_0$ (or higher excited states) to approach each other closely in energy. 
For instance, a common motif involves a torsion around the C–N bond coupled with pyramidalization at carbon.

These coordinates are not just theoretical curiosities—they are directly connected to actual relaxation pathways that the molecule follows after photoexcitation.

In [ ]:
%matplotlib widget

In [ ]:
# loading the ground truth MRCI data for the configurations above

data_grid = np.load("groundtruth_grid.npz")
data_grid.keys()

In [ ]:
bond_values = np.linspace(2.4321, 4.4321, 101)
angle_values = np.linspace(0, 90, 91)
X, Y = np.meshgrid(angle_values, bond_values)

In [ ]:
mask = data_grid['energy'][:,:,0] < -1 # geometries where the SCF did not converge

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

for state in range(3):
    Z = data_grid['energy'][:, :, state]
    surf = ax.plot_trisurf(X[mask].flatten(), Y[mask].flatten(), Z[mask].flatten(), cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)

# Z = results_model_comp[model_type]['energies'][:,2]#.reshape(targets_bond.shape[0], targets_angle.shape[0])
# surf = ax.plot_trisurf(X, Y, Z, cmap=cm.coolwarm,
#                        linewidth=0, antialiased=False)

ax.set_ylabel("C-N bond length / bohr")
ax.set_xlabel("H-C-N-H angle / degree")
ax.set_zlabel("Energy / Ha")
plt.show()